# Week 9 Lab: Mini API Client with OOP

This notebook walks you through building a tiny API client using Python classes and the Cat Facts API. Work through each checkpoint, run the tests, and annotate the code with comments as you go.

## Learning Objectives
- Translate beginner OOP vocabulary (class, object, attribute, method, constructor) into working Python code.
- Send HTTP requests with `requests`, inspect the response, and handle errors safely.
- Encapsulate API logic inside a reusable class that tracks state and provides helper methods.
- Practice debugging by reading stack traces, printing intermediate values, and writing assertions.

## Getting Ready
1. **Run the next cell first** to import Python libraries used in the lab.
2. If you see an error about `requests` not being found, open a terminal and install it with `pip install requests`.
3. Create a `data/` folder next to this notebook if you plan to save files locally.

In [ ]:
import json
from datetime import datetime
from pathlib import Path

import requests

print(f"requests version: {requests.__version__}")

## Warm-Up: Explore a Finished Class
Before writing your own class, let's review what a simple finished version looks like. Run the next cell and read the comments so you can map each OOP vocabulary term to real code.


In [ ]:
class Playlist:
    """Represent a tiny music playlist."""

    def __init__(self, title, songs=None):
        # Constructor: runs when we make a new Playlist.
        self.title = title  # attribute
        self.songs = songs or []  # attribute

    def add_song(self, song_title):
        """Method that updates the object's state."""
        self.songs.append(song_title)

    def summary(self):
        return f'Playlist {self.title} has {len(self.songs)} songs.'

study_mix = Playlist('Study Session', songs=['Lofi Sunrise'])
study_mix.add_song('Rainy Beats')
print(study_mix.summary())
print('Songs stored:', study_mix.songs)


### What to Notice
1. `Playlist` is the **class** (the blueprint).
2. `study_mix` is an **object** created from that class.
3. Inside `__init__`, we set up our **attributes** (`title`, `songs`).
4. Methods like `add_song` and `summary` let the object do useful actions.

Use this as a reference when you build your own class below. If you get stuck later, rerun this cell and compare.


## Checkpoint 1 – Practice With Classes
Now that you've seen a complete example, it's your turn to finish a small class. Complete the `summary` method so it returns a friendly string. Run the test cell below to make sure the class works. Extra challenge: add an optional `milk_option` when creating the order.

**Tip:** Keep the warm-up example open in another window or re-run it whenever you want a reminder of how attributes and methods connect.


In [ ]:
class CoffeeOrder:
    """Blueprint for a coffee order."""

    def __init__(self, size, drink_type, milk_option='whole'):
        self.size = size
        self.drink_type = drink_type
        self.milk_option = milk_option

    def summary(self):
        """Return a human-readable description of the order.

        Step-by-step hints:
        1. Build a base string with the drink size and type (consider `.title()` to capitalize).
        2. Mention the milk option (if there is one).
        3. Return the finished sentence.
        """
        raise NotImplementedError('Implement summary to describe the coffee order')


In [ ]:
# ✅ Test your CoffeeOrder class here
# Need a reminder? Scroll up to rerun the Playlist example before trying this test.
sample = CoffeeOrder('medium', 'latte', milk_option='oat')
assert sample.size == 'medium'
assert 'latte' in sample.drink_type

try:
    preview = sample.summary()
except NotImplementedError:
    raise AssertionError('summary() still needs to be implemented. Follow the TODO in the class above.')
else:
    assert isinstance(preview, str)
    assert 'oat' in preview
    print('Great! summary() is returning:', preview)


## Checkpoint 2 – Explore an API Response
Run the cell to make a GET request. Investigate the response dictionary to find the keys you might want to store on your object.

In [ ]:
CAT_FACT_URL = "https://catfact.ninja/fact"

response = requests.get(CAT_FACT_URL, params={"max_length": 80})
print("Status:", response.status_code)
print("Headers:", response.headers.get("Content-Type"))

fact_payload = response.json()
print("Keys returned:", list(fact_payload.keys()))
print("Example fact:
", fact_payload["fact"])

## Checkpoint 3 – Build the `CatFactsClient`
The skeleton class below wraps the Cat Facts API. Follow the TODO comments to finish each method. Aim to:
- Store the most recent fact in `self.last_fact` and the timestamp in `self.last_fetched_at`.
- Reuse `_handle_response` so you have consistent error messages.
- Implement `save_last_fact` to write the last fact to `data/last_fact.json` (or another path you choose).
- Fill in `search_facts` so it filters results containing a keyword.

Run the tests afterwards to confirm everything works. When a test fails, read the assertion message and debug step-by-step.

In [ ]:
class CatFactsClient:
    """Small API wrapper that remembers the most recent fact."""

    BASE_URL = "https://catfact.ninja"

    def __init__(self, session=None):
        # Allow passing in a requests.Session for easier testing or caching.
        self.session = session or requests.Session()
        self.last_fact = None
        self.last_fetched_at = None

    def _handle_response(self, response):
        """Raise helpful errors and return the parsed JSON payload."""
        try:
            response.raise_for_status()
        except requests.HTTPError as exc:
            status = response.status_code
            snippet = response.text[:200]
            raise RuntimeError(f"API error {status}: {snippet}") from exc
        return response.json()

    def random_fact(self, max_length=120):
        """Fetch a single random fact and remember when it was fetched."""
        url = f"{self.BASE_URL}/fact"
        payload = self._handle_response(
            self.session.get(url, params={"max_length": max_length})
        )
        # TODO: store the fact text and timestamp on the instance.
        raise NotImplementedError("Store the fact and timestamp, then return the fact text")

    def facts_batch(self, limit=3):
        """Return a list of fact strings using the bulk endpoint."""
        url = f"{self.BASE_URL}/facts"
        payload = self._handle_response(self.session.get(url, params={"limit": limit}))
        # TODO: extract only the "fact" field from each item in payload["data"].
        raise NotImplementedError("Return a list of fact strings from the API response")

    def save_last_fact(self, path=Path("data/last_fact.json")):
        """Write the last fetched fact and timestamp to disk.

        Tip: use json.dump with a dictionary that includes both pieces of data.
        If no fact has been fetched yet, raise a ValueError with a helpful message.
        """
        raise NotImplementedError("Save the last fact to JSON. Check that last_fact exists first.")

    def search_facts(self, keyword, limit=10):
        """Return only the facts that contain the keyword (case-insensitive)."""
        if not keyword:
            raise ValueError("Please provide a keyword to search for.")
        # TODO: fetch a batch and filter results based on the keyword.
        raise NotImplementedError("Filter the batch results to those containing the keyword")

In [ ]:
# ✅ Tests for CatFactsClient
client = CatFactsClient()

try:
    fact = client.random_fact(max_length=60)
except NotImplementedError:
    raise AssertionError("Implement random_fact before running the tests.")
else:
    assert isinstance(fact, str) and fact, "random_fact should return a non-empty string"
    assert client.last_fact == fact, "Store the fact on last_fact"
    assert client.last_fetched_at is not None, "Remember when the fact was fetched"
    print("Random fact:", fact)

try:
    batch = client.facts_batch(limit=5)
except NotImplementedError:
    raise AssertionError("Implement facts_batch before running the tests.")
else:
    assert isinstance(batch, list) and batch, "facts_batch should return a list"
    assert all(isinstance(item, str) for item in batch), "Each batch item should be a string"
    print("Batch length:", len(batch))

client.last_fact = "Cats have retractable claws."
client.last_fetched_at = datetime(2024, 1, 3, 12, 0, 0)
try:
    client.save_last_fact(path=Path("last_fact_preview.json"))
except NotImplementedError:
    raise AssertionError("Complete save_last_fact so it writes JSON.")
else:
    preview = Path("last_fact_preview.json")
    assert preview.exists(), "The JSON file was not created. Check the path."
    content = json.loads(preview.read_text())
    assert "fact" in content and "fetched_at" in content
    preview.unlink()  # clean up temporary file
    print("Saved fact preview:", content)

try:
    results = client.search_facts("cat", limit=5)
except NotImplementedError:
    raise AssertionError("Implement search_facts to filter facts by keyword.")
else:
    assert all("cat" in fact.lower() for fact in results), "Each result should include the keyword"
    print("Filtered results count:", len(results))

## Extension Ideas
- Add a method `random_fact_with_length(self, min_length, max_length)` that loops until it finds a fact inside the range.
- Store fetched facts in a list attribute and create a method to list unique words.
- Build a simple command-line menu using `input()` that calls the client methods.
- Swap in another API (Dog Facts, Bored API) and subclass `CatFactsClient` to reuse logic.

Celebrate once all tests pass and commit your work!